## Set dataloader

### Important parameter
**`mode`** :'train' or 'test'  (`mode='train'`...)  
**`batch_size`** :set batch size  
**`vocab_threshold`** :How many times word to set a keys for that words  
**`vocab_from_file`** - a Boolean that decides whether to load the vocabulary from file.  

In [6]:
!pip install pycocotools

In [7]:
import sys
sys.path.append('C:/Users/Da/Desktop/image-captioning/cocoapi/PythonAPI')
!pip install nltk
import nltk
nltk.download('punkt')
from data_loader import get_loader
from torchvision import transforms
from pycocotools.coco import COCO

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Da\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
# Define a transform to pre-process the training images.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Set the minimum word count threshold.
vocab_threshold = 5

# Specify the batch size.
batch_size = 500

# load annotations and transform word to numbers first time
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=False)

#data_loader.dataset.caption_lengths

loading annotations into memory...
Done (t=0.59s)
creating index...
index created!
[0/414113] Tokenizing captions...
[100000/414113] Tokenizing captions...
[200000/414113] Tokenizing captions...
[300000/414113] Tokenizing captions...
[400000/414113] Tokenizing captions...
loading annotations into memory...


  1%|▍                                                                        | 2585/414113 [00:00<00:31, 12961.03it/s]

Done (t=0.60s)
creating index...
index created!
Obtaining caption lengths...


100%|███████████████████████████████████████████████████████████████████████| 414113/414113 [00:32<00:00, 12825.59it/s]


## Define start and end word values

In [9]:
import nltk
#divide sample sentence
sample_caption = 'A person doing a trick on a rail while riding a skateboard.'
sample_tokens = nltk.tokenize.word_tokenize(str(sample_caption).lower())
print(sample_tokens)

['a', 'person', 'doing', 'a', 'trick', 'on', 'a', 'rail', 'while', 'riding', 'a', 'skateboard', '.']


In [12]:
#define start word always is integer 0
sample_caption = []

start_word = data_loader.dataset.vocab.start_word
print('Special start word:', start_word)
sample_caption.append(data_loader.dataset.vocab(start_word))
print(sample_caption[0])

Special start word: <start>
0


In [13]:
#define end word always always is integer 1
end_word = data_loader.dataset.vocab.end_word
print('Special end word:', end_word)

sample_caption.append(data_loader.dataset.vocab(end_word))
print(sample_caption[-1])

Special end word: <end>
1


In [14]:
import torch
#check the start and end word value
sample_caption = torch.Tensor(sample_caption).long()
print(sample_caption)

tensor([0, 1])


In [14]:
# Preview the word2idx dictionary.
#dict(list(data_loader.dataset.vocab.word2idx.items())[:10])

{'<start>': 0,
 '<end>': 1,
 '<unk>': 2,
 'a': 3,
 'very': 4,
 'clean': 5,
 'and': 6,
 'well': 7,
 'decorated': 8,
 'empty': 9}

In [15]:
# Print the total number of keys in the word2idx dictionary.
#print('Total number of tokens in vocabulary:', len(data_loader.dataset.vocab))

Total number of tokens in vocabulary: 8852


In [16]:
vocab_threshold = 4
# change parameter for data_loader
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=False)

loading annotations into memory...
Done (t=0.58s)
creating index...
index created!
[0/414113] Tokenizing captions...
[100000/414113] Tokenizing captions...
[200000/414113] Tokenizing captions...
[300000/414113] Tokenizing captions...
[400000/414113] Tokenizing captions...
loading annotations into memory...
Done (t=0.57s)
creating index...
index created!
Obtaining caption lengths...

  0%|▏                                                                        | 1318/414113 [00:00<00:31, 13179.89it/s]

100%|███████████████████████████████████████████████████████████████████████| 414113/414113 [00:32<00:00, 12879.02it/s]


In [17]:
# Print the total number of keys in the word2idx dictionary.
print('Total number of tokens in vocabulary:', len(data_loader.dataset.vocab))

Total number of tokens in vocabulary: 9947


## Set unkown words to the integer `2`.

In [18]:
unk_word = data_loader.dataset.vocab.unk_word
#print('Special unknown word:', unk_word)
#print('All unknown words are mapped to this integer:', data_loader.dataset.vocab(unk_word))

Special unknown word: <unk>
All unknown words are mapped to this integer: 2


In [19]:
#cheak the unkown word values
#print(data_loader.dataset.vocab('jfkafejw'))
#print(data_loader.dataset.vocab('ieowoqjf'))

2
2


## Load the past version data loader:  
  from file:vocab.pkl.(Like yesterday or other days set words to values dictionary)
### if `vocab_from_file=True`, then any supplied argument for `vocab_threshold` when instantiating the data loader is completely ignored.

In [19]:
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_from_file=True)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=0.58s)
creating index...


  0%|▏                                                                        | 1182/414113 [00:00<00:35, 11531.71it/s]

index created!
Obtaining caption lengths...


100%|███████████████████████████████████████████████████████████████████████| 414113/414113 [00:31<00:00, 12943.12it/s]


In [20]:
#Look dictionary values counts

from collections import Counter
# Tally the total number of training captions with each length.
counter = Counter(data_loader.dataset.caption_lengths)
lengths = sorted(counter.items(), key=lambda pair: pair[1], reverse=True)
for value, count in lengths:
    print('value: %2d --- count: %5d' % (value, count))

value: 10 --- count: 86302
value: 11 --- count: 79971
value:  9 --- count: 71920
value: 12 --- count: 57653
value: 13 --- count: 37668
value: 14 --- count: 22342
value:  8 --- count: 20742
value: 15 --- count: 12839
value: 16 --- count:  7736
value: 17 --- count:  4845
value: 18 --- count:  3101
value: 19 --- count:  2017
value:  7 --- count:  1594
value: 20 --- count:  1453
value: 21 --- count:   997
value: 22 --- count:   683
value: 23 --- count:   534
value: 24 --- count:   384
value: 25 --- count:   277
value: 26 --- count:   214
value: 27 --- count:   160
value: 28 --- count:   114
value: 29 --- count:    87
value: 30 --- count:    58
value: 31 --- count:    49
value: 32 --- count:    44
value: 34 --- count:    40
value: 37 --- count:    32
value: 35 --- count:    31
value: 33 --- count:    30
value: 36 --- count:    26
value: 38 --- count:    18
value: 39 --- count:    18
value: 43 --- count:    16
value: 44 --- count:    16
value: 48 --- count:    12
value: 45 --- count:    11
v